In [1]:
import snn_utils
import torch.nn as nn
from torch.nn import functional as F
import torch 
import numpy as np
import pylab as plt
import snn_delay_utils
import snn_IPSP_utils
import snn_IPSP_delay_utils
import random
from os.path import dirname, realpath
device = 'cpu'

print(torch.__version__)

from collections import namedtuple

ModuleNotFoundError: No module named 'snn_utils'

## Sequence Visualizer

El proposito de esta notebook es el poder graficar tanto la entrada como salida de la red DSNN que reconoce patrones en una secuencia de numeros aleatorios.

In [ ]:
Sin_tmp = np.load('./sequences/seq_0.npy')
print(Sin_tmp.shape)
Sin = np.reshape(Sin_tmp, (Sin_tmp.shape[0]*Sin_tmp.shape[1], Sin_tmp.shape[2]))
Sin = np.transpose(Sin)
print(Sin.shape)

In [ ]:
snn_utils.plotLIF(U=None, S=Sin)
np.count_nonzero(Sin)